###### Importing libraries

In [118]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
% matplotlib inline

In [119]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


###### Reading in the data to model

In [120]:
X_test = pd.read_csv('../Data/test_x.csv')
X = pd.read_csv('../Data/train_x.csv')
y = pd.read_csv('../Data/train_y.csv', header = None)
print(X_test.shape)
print(X.shape)
print(y.shape)

(1459, 111)
(1445, 111)
(1445, 1)


###### Gradient Boosting (book, p. 198)

In [122]:
# http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html

In [123]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X, np.ravel(y))

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=300, random_state=42)
gbrt.fit(X_train, y_train)

errors = [mean_squared_error(y_val, y_pred)
          for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors)

gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators, random_state=42)
gbrt_best.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=2, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=295, presort='auto', random_state=42,
             subsample=1.0, verbose=0, warm_start=False)

In [125]:
y_val.shape

(362,)

In [126]:
bst_n_estimators  # 270 (OUT OF 300)

295

In [127]:
min(errors) # 0.014635885043588544

0.012909797755950225

In [128]:
min(errors)**0.5

0.11362129094474427

In [129]:
errors[295]**0.5  # 0.12097886

0.11362129094474427

###### Early Stopping

In [130]:
# Doesn't work well - error is too large
gbrt = GradientBoostingRegressor(max_depth=4, warm_start=True, random_state=42)

min_val_error = float("inf")
error_going_up = 0
for n_estimators in range(1, 300):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train, y_train)
    y_pred = gbrt.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred)
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up = 0
    else:
        error_going_up += 1
        if error_going_up == 5:
            break  # early stopping

In [132]:
min_val_error**0.5

0.16982096466147564

In [133]:
n_estimators

16